# M1 NO FAULT

IMPORTAZIONE LIBRERIE

In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.signal as signal
from scipy.fft import fft, fftfreq
from scipy.signal import find_peaks
import math

CAMBIO PATH PER ANALISI RISPETTO AL LOG DI VOLO CONSIDERATO

In [2]:
path_file = os.path.abspath(os.getcwd())
print(path_file)
os.chdir(path_file)

c:\Users\chiar\OneDrive\Desktop\MantenzionePreventiva\ProgettoManutenzionePreventiva\M1\NO_FAULT1


SETTAGGIO DELLA VARIABILE RELATIVA AL GUASTO

In [3]:
guasto = 0

## RCOU

VARIABILE RCOU POTENZA MOTORI.

Andiamo a prendere una potenza media di 1350 per considerare il momento il cui inizia/finisce il volo

In [4]:
rcou_m1 = pd.read_csv("RCOU.csv")
rcou_m1 = rcou_m1.astype("float64")
rcou_m1 = rcou_m1.drop(['LineNo','C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8'], axis=1)

potenza = 1350

rcou_m1 = rcou_m1[((rcou_m1['C9'] >= potenza) & (rcou_m1['C10'] >= potenza) & (rcou_m1['C11'] >= potenza) & (rcou_m1['C12'] >= potenza) & (rcou_m1['C13'] >= potenza) & (rcou_m1['C14']>= potenza))]
rcou_m1=rcou_m1.reset_index(drop=True)

display(rcou_m1)

,TimeUS,C9,C10,C11,C12,C13,C14
0,22664152.0,1360.0,1375.0,1382.0,1352.0,1371.0,1363.0
1,22676651.0,1382.0,1369.0,1360.0,1390.0,1364.0,1387.0
2,22771304.0,1452.0,1411.0,1456.0,1406.0,1473.0,1386.0
3,22782035.0,1507.0,1358.0,1409.0,1465.0,1479.0,1393.0
4,22792632.0,1519.0,1355.0,1384.0,1497.0,1465.0,1420.0
...,...,...,...,...,...,...,...
4180,68488846.0,1553.0,1460.0,1514.0,1503.0,1422.0,1584.0
4181,68499144.0,1536.0,1469.0,1506.0,1501.0,1397.0,1594.0
4182,68509358.0,1542.0,1448.0,1511.0,1482.0,1419.0,1565.0
4183,68523070.0,1572.0,1386.0,1463.0,1509.0,1415.0,1550.0


Andiamo a prendere i tempi di inizio e di fine, così da poter effettuare il trimming delle altre tabelle

In [5]:
min=rcou_m1['TimeUS'][0]
max=rcou_m1['TimeUS'][len(rcou_m1)-1]
print(max)
print(min)

68533079.0
22664152.0


## ATT & XFK

### XFK

XFK utilizzato per fare poi il confronto con i valori di roll, pitch e yaw presenti att

In [6]:
xkf1_m1 = pd.read_csv("XKF1_0.csv")
xkf1_m1 = xkf1_m1.astype("float64")
xkf1_m1 = xkf1_m1.drop(['LineNo','C','VN','VE','VD','dPD','PN','PE','PD','GX','GY','GZ','OH'], axis=1)
xkf1_m1 = xkf1_m1[((xkf1_m1['TimeUS'] >= min) & (xkf1_m1['TimeUS'] <= max))]
xkf1_m1 = xkf1_m1.reset_index(drop=True)
print(xkf1_m1)

          TimeUS  Roll  Pitch    Yaw
0     22675051.0  0.01  -0.44  13.24
1     22685153.0  0.03  -0.40  13.24
2     22695762.0 -0.03  -0.45  13.23
3     22705793.0 -0.04  -0.52  13.22
4     22715779.0  0.03  -0.40  13.23
...          ...   ...    ...    ...
4256  68486359.0 -4.46   1.19   5.63
4257  68496743.0 -4.40   1.35   5.58
4258  68507280.0 -4.34   1.52   5.53
4259  68521132.0 -4.14   1.73   5.49
4260  68531134.0 -3.95   1.91   5.45

[4261 rows x 4 columns]


### ATT

Andiamo a prendere dal csv relativo ad ATT i dati relativi alle variabili di roll, pitch e yaw

In [7]:
att_m1 = pd.read_csv("ATT.csv")
att_m1 = att_m1.astype("float64")
att_m1 = att_m1.drop(['LineNo','ErrRP', 'ErrYaw', 'AEKF'], axis=1)
att_m1 = att_m1[((att_m1['TimeUS'] >= min) & (att_m1['TimeUS'] <= max))]
att_m1 = att_m1.reset_index(drop=True)
print(att_m1)

          TimeUS  DesRoll  Roll  DesPitch  Pitch  DesYaw    Yaw
0     22675940.0     0.00  0.01      0.00  -0.44   13.23  13.24
1     22686733.0     0.00  0.03      0.00  -0.40   13.23  13.24
2     22696746.0     0.00 -0.03      0.00  -0.45   13.23  13.23
3     22706870.0     0.00 -0.04      0.00  -0.52   13.23  13.22
4     22716835.0     0.00  0.03      0.00  -0.40   13.23  13.23
...          ...      ...   ...       ...    ...     ...    ...
4256  68488238.0    -1.70 -4.46     -0.22   1.19    5.99   5.63
4257  68498617.0    -1.59 -4.40     -0.21   1.35    5.99   5.58
4258  68508630.0    -1.49 -4.34     -0.20   1.52    5.99   5.53
4259  68522298.0    -1.39 -4.14     -0.18   1.73    5.99   5.49
4260  68532571.0    -1.30 -3.95     -0.17   1.91    5.99   5.45

[4261 rows x 7 columns]


Tramite un ciclo for si va a vedere se i valori di XFK corrispondono con quelli di ATT in caso contrario fa una media tra i due

In [8]:
for idx,i in enumerate(att_m1['Roll']):
    if(xkf1_m1['Roll'][idx] != i):
        att_m1['Roll'][idx] = ((att_m1['Roll'][idx] + xkf1_m1['Roll'][idx])/2)

for idx,i in enumerate(att_m1['Pitch']):
    if(xkf1_m1['Pitch'][idx] != i):
        att_m1['Pitch'][idx] = ((att_m1['Pitch'][idx] + xkf1_m1['Pitch'][idx])/2)

for idx,i in enumerate(att_m1['Yaw']):
    if(xkf1_m1['Yaw'][idx] != i):
        att_m1['Yaw'][idx] = ((att_m1['Yaw'][idx] + xkf1_m1['Yaw'][idx])/2)

print(att_m1)

          TimeUS  DesRoll  Roll  DesPitch  Pitch  DesYaw    Yaw
0     22675940.0     0.00  0.01      0.00  -0.44   13.23  13.24
1     22686733.0     0.00  0.03      0.00  -0.40   13.23  13.24
2     22696746.0     0.00 -0.03      0.00  -0.45   13.23  13.23
3     22706870.0     0.00 -0.04      0.00  -0.52   13.23  13.22
4     22716835.0     0.00  0.03      0.00  -0.40   13.23  13.23
...          ...      ...   ...       ...    ...     ...    ...
4256  68488238.0    -1.70 -4.46     -0.22   1.19    5.99   5.63
4257  68498617.0    -1.59 -4.40     -0.21   1.35    5.99   5.58
4258  68508630.0    -1.49 -4.34     -0.20   1.52    5.99   5.53
4259  68522298.0    -1.39 -4.14     -0.18   1.73    5.99   5.49
4260  68532571.0    -1.30 -3.95     -0.17   1.91    5.99   5.45

[4261 rows x 7 columns]


# IMU

VALORI DI ACCELERAZIONE E DEL GIROSCOPIO

Per le 3 tabelle di IMU si sono presi i soli valori relativi a GyrX, GyrY, GyrZ, AccX, AccY, AccZ.
In seguito si è poi creata una singola tabella popolandola con la media dei valori delle tre tabelle.

In [10]:
imu_0_m1 = pd.read_csv("IMU_0.csv")
imu_0_m1 = imu_0_m1.astype("float64")
imu_0_m1 = imu_0_m1.drop(['LineNo','I', 'EG', 'EA', 'T', 'GH', 'AH', 'GHz', 'AHz'], axis=1)
imu_0_m1 = imu_0_m1[((imu_0_m1['TimeUS'] >= min) & (imu_0_m1['TimeUS'] <= max))]

imu_1_m1 = pd.read_csv("IMU_1.csv")
imu_1_m1 = imu_1_m1.astype("float64")
imu_1_m1 = imu_1_m1.drop(['LineNo','I', 'EG', 'EA', 'T', 'GH', 'AH', 'GHz', 'AHz'], axis=1)
imu_1_m1 = imu_1_m1[((imu_1_m1['TimeUS'] >= min) & (imu_1_m1['TimeUS'] <= max))]

imu_2_m1 = pd.read_csv("IMU_2.csv")
imu_2_m1 = imu_2_m1.astype("float64")
imu_2_m1 = imu_2_m1.drop(['LineNo','I', 'EG', 'EA', 'T', 'GH', 'AH', 'GHz', 'AHz'], axis=1)
imu_2_m1 = imu_2_m1[((imu_2_m1['TimeUS'] >= min) & (imu_2_m1['TimeUS'] <= max))]

imu_m1 = pd.concat((imu_0_m1, imu_1_m1, imu_2_m1))
imu_m1=imu_m1.groupby(imu_m1.TimeUS, as_index=False).mean()

display(imu_m1)


,TimeUS,GyrX,GyrY,GyrZ,AccX,AccY,AccZ
0,22666080.0,-0.000558,-0.072877,-0.005863,-0.278756,0.024499,-9.481081
1,22668252.0,0.014605,-0.051462,-0.010475,-0.311486,0.012792,-9.464686
2,22670635.0,0.042057,0.029661,-0.001371,-0.379617,0.080195,-9.710578
3,22675772.0,0.044220,0.056586,0.002729,-0.318851,0.121676,-9.868946
4,22681012.0,-0.035033,-0.112585,-0.025552,-0.159212,0.062739,-9.546370
...,...,...,...,...,...,...,...
16503,68522130.0,0.269569,0.293996,-0.016782,-0.722770,2.779444,-11.869947
16504,68524886.0,0.331998,0.249299,-0.035411,-0.695731,2.937416,-12.088990
16505,68527023.0,0.337875,0.270059,-0.040891,-0.844330,2.975086,-12.301677
16506,68530131.0,0.314659,0.326277,-0.025974,-0.837869,3.015614,-12.519287


## UNIONE DELLE TABELLE

Vengono unite le tabelle relative a IMU e ATT, riempiendo i valori mancanti con quelli vicini

In [11]:
m1_nofault = pd.merge_ordered(imu_m1,att_m1)
m1_nofault=m1_nofault.fillna(method="ffill").fillna(method="bfill")
print(m1_nofault)

           TimeUS      GyrX      GyrY      GyrZ      AccX      AccY  \
0      22666080.0 -0.000558 -0.072877 -0.005863 -0.278756  0.024499   
1      22668252.0  0.014605 -0.051462 -0.010475 -0.311486  0.012792   
2      22670635.0  0.042057  0.029661 -0.001371 -0.379617  0.080195   
3      22675772.0  0.044220  0.056586  0.002729 -0.318851  0.121676   
4      22675940.0  0.044220  0.056586  0.002729 -0.318851  0.121676   
...           ...       ...       ...       ...       ...       ...   
20764  68524886.0  0.331998  0.249299 -0.035411 -0.695731  2.937416   
20765  68527023.0  0.337875  0.270059 -0.040891 -0.844330  2.975086   
20766  68530131.0  0.314659  0.326277 -0.025974 -0.837869  3.015614   
20767  68532323.0  0.326869  0.320070 -0.022980 -0.659711  3.074340   
20768  68532571.0  0.326869  0.320070 -0.022980 -0.659711  3.074340   

            AccZ  DesRoll  Roll  DesPitch  Pitch  DesYaw    Yaw  
0      -9.481081     0.00  0.01      0.00  -0.44   13.23  13.24  
1      -9.46468

Alla tabella risultante dal procedimento sopra, viene unita la tabella RCOU, riempiendo i valori mancanti con quelli vicini

In [12]:
m1_nofault = pd.merge_ordered(m1_nofault, rcou_m1)
m1_nofault=m1_nofault.fillna(method="ffill").fillna(method="bfill")

print(m1_nofault)

           TimeUS      GyrX      GyrY      GyrZ      AccX      AccY  \
0      22664152.0 -0.000558 -0.072877 -0.005863 -0.278756  0.024499   
1      22666080.0 -0.000558 -0.072877 -0.005863 -0.278756  0.024499   
2      22668252.0  0.014605 -0.051462 -0.010475 -0.311486  0.012792   
3      22670635.0  0.042057  0.029661 -0.001371 -0.379617  0.080195   
4      22675772.0  0.044220  0.056586  0.002729 -0.318851  0.121676   
...           ...       ...       ...       ...       ...       ...   
24949  68527023.0  0.337875  0.270059 -0.040891 -0.844330  2.975086   
24950  68530131.0  0.314659  0.326277 -0.025974 -0.837869  3.015614   
24951  68532323.0  0.326869  0.320070 -0.022980 -0.659711  3.074340   
24952  68532571.0  0.326869  0.320070 -0.022980 -0.659711  3.074340   
24953  68533079.0  0.326869  0.320070 -0.022980 -0.659711  3.074340   

            AccZ  DesRoll  Roll  DesPitch  Pitch  DesYaw    Yaw      C9  \
0      -9.481081     0.00  0.01      0.00  -0.44   13.23  13.24  1360.0 

Vengono aggiornati i tempi facendo partire il volo all'istante 0, e viene inserita la colonna relativa al guasto, valorizzandola con la variabile *guasto*.

In [13]:
m1_nofault["TimeUS"] = m1_nofault["TimeUS"] - m1_nofault.iloc[0]["TimeUS"]
m1_nofault["TimeUS"] = m1_nofault["TimeUS"].astype(int)
m1_nofault["Guasto"] = guasto
print(m1_nofault)

         TimeUS      GyrX      GyrY      GyrZ      AccX      AccY       AccZ  \
0             0 -0.000558 -0.072877 -0.005863 -0.278756  0.024499  -9.481081   
1          1928 -0.000558 -0.072877 -0.005863 -0.278756  0.024499  -9.481081   
2          4100  0.014605 -0.051462 -0.010475 -0.311486  0.012792  -9.464686   
3          6483  0.042057  0.029661 -0.001371 -0.379617  0.080195  -9.710578   
4         11620  0.044220  0.056586  0.002729 -0.318851  0.121676  -9.868946   
...         ...       ...       ...       ...       ...       ...        ...   
24949  45862871  0.337875  0.270059 -0.040891 -0.844330  2.975086 -12.301677   
24950  45865979  0.314659  0.326277 -0.025974 -0.837869  3.015614 -12.519287   
24951  45868171  0.326869  0.320070 -0.022980 -0.659711  3.074340 -12.657953   
24952  45868419  0.326869  0.320070 -0.022980 -0.659711  3.074340 -12.657953   
24953  45868927  0.326869  0.320070 -0.022980 -0.659711  3.074340 -12.657953   

       DesRoll  Roll  DesPitch  Pitch  

In [ ]:
last_cell = m1_nofault.iloc[-1, m1_nofault.columns.get_loc('TimeUS')]
to_be=[]
for i in range(0, last_cell,2857):
    to_be.append(i)
df = pd.DataFrame(to_be, columns = ['TimeUS'])

Viene corretta la frequenza di campionamento in modo tale da avere 350 misurazioni per ogni secondo

In [17]:
m1_nofault_final = pd.merge_ordered(m1_nofault,df)
m1_nofault_final=m1_nofault_final.fillna(method="ffill").fillna(method="bfill")
m1_nofault_final = m1_nofault_final[m1_nofault_final.TimeUS.isin(to_be)]

         TimeUS      GyrX      GyrY      GyrZ      AccX      AccY       AccZ  \
0             0 -0.000558 -0.072877 -0.005863 -0.278756  0.024499  -9.481081   
1          1928 -0.000558 -0.072877 -0.005863 -0.278756  0.024499  -9.481081   
2          2857 -0.000558 -0.072877 -0.005863 -0.278756  0.024499  -9.481081   
3          4100  0.014605 -0.051462 -0.010475 -0.311486  0.012792  -9.464686   
4          5714  0.014605 -0.051462 -0.010475 -0.311486  0.012792  -9.464686   
...         ...       ...       ...       ...       ...       ...        ...   
40996  45865979  0.314659  0.326277 -0.025974 -0.837869  3.015614 -12.519287   
40997  45866278  0.314659  0.326277 -0.025974 -0.837869  3.015614 -12.519287   
40998  45868171  0.326869  0.320070 -0.022980 -0.659711  3.074340 -12.657953   
40999  45868419  0.326869  0.320070 -0.022980 -0.659711  3.074340 -12.657953   
41000  45868927  0.326869  0.320070 -0.022980 -0.659711  3.074340 -12.657953   

       DesRoll  Roll  DesPitch  Pitch  

Viene formattata la colonna del tempo in maniera tale che sia più leggibile all'operatore umano

In [20]:
from datetime import timedelta
m1_nofault_final=m1_nofault_final.reset_index(drop=True)
m1_nofault_final['TimeUS'] = pd.to_datetime(m1_nofault_final['TimeUS'], unit='us').dt.strftime('%H:%M:%S.%f')
display(m1_nofault_final)

,TimeUS,GyrX,GyrY,GyrZ,AccX,AccY,AccZ,DesRoll,Roll,DesPitch,Pitch,DesYaw,Yaw,C9,C10,C11,C12,C13,C14,Guasto
0,00:00:00.000000,-0.000558,-0.072877,-0.005863,-0.278756,0.024499,-9.481081,0.00,0.01,0.00,-0.44,13.23,13.24,1360.0,1375.0,1382.0,1352.0,1371.0,1363.0,0.0
1,00:00:00.002857,-0.000558,-0.072877,-0.005863,-0.278756,0.024499,-9.481081,0.00,0.01,0.00,-0.44,13.23,13.24,1360.0,1375.0,1382.0,1352.0,1371.0,1363.0,0.0
2,00:00:00.005714,0.014605,-0.051462,-0.010475,-0.311486,0.012792,-9.464686,0.00,0.01,0.00,-0.44,13.23,13.24,1360.0,1375.0,1382.0,1352.0,1371.0,1363.0,0.0
3,00:00:00.008571,0.042057,0.029661,-0.001371,-0.379617,0.080195,-9.710578,0.00,0.01,0.00,-0.44,13.23,13.24,1360.0,1375.0,1382.0,1352.0,1371.0,1363.0,0.0
4,00:00:00.011428,0.042057,0.029661,-0.001371,-0.379617,0.080195,-9.710578,0.00,0.01,0.00,-0.44,13.23,13.24,1360.0,1375.0,1382.0,1352.0,1371.0,1363.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16050,00:00:45.854850,0.229661,0.255619,-0.026397,-0.642934,2.202350,-11.467943,-1.49,-4.34,-0.20,1.52,5.99,5.53,1542.0,1448.0,1511.0,1482.0,1419.0,1565.0,0.0
16051,00:00:45.857707,0.229661,0.255619,-0.026397,-0.642934,2.202350,-11.467943,-1.49,-4.34,-0.20,1.52,5.99,5.53,1542.0,1448.0,1511.0,1482.0,1419.0,1565.0,0.0
16052,00:00:45.860564,0.269569,0.293996,-0.016782,-0.722770,2.779444,-11.869947,-1.39,-4.14,-0.18,1.73,5.99,5.49,1572.0,1386.0,1463.0,1509.0,1415.0,1550.0,0.0
16053,00:00:45.863421,0.337875,0.270059,-0.040891,-0.844330,2.975086,-12.301677,-1.39,-4.14,-0.18,1.73,5.99,5.49,1572.0,1386.0,1463.0,1509.0,1415.0,1550.0,0.0


## Analisi nel tempo e in frequenza

Funzione che calcola il Root Mean Square

In [21]:
import math
#Funzione che calcola il Root Mean Square (RMS)
def rmsValue(arr, n):
    square = 0
    mean = 0.0
    root = 0.0
    #Calcola il quadrato
    for i in range(0,n):
        square += (arr[i]**2)
    #Calcola la media
    mean = (square / (float)(n))
    #Calcola la radice
    root = math.sqrt(mean)
    return root

Viene definita la funzione generica per ottenere la tabellina con le feauture nel tempo e in frequenza


In [22]:
def time_freq_feat(V):
    N=350
    height_threshold=0.0
    tab= pd.DataFrame([], columns=['tempo',f'<>({V})',f'var({V})',f'kurt({V})',f'RMS({V})',f'freq1({V})',f'freq2({V})',f'amp1({V})',f'amp2({V})'])
    for i in range(int(len(m1_nofault_final)/350)):
        V1=m1_nofault_final[str(V)].iloc[:(i+1*350)+1]
        mean=V1.mean()
        var=V1.var()
        kurt=V1.kurtosis()
        rms=rmsValue(V1,len(V1))
        yf=fft(V1.values)
        xf=fftfreq(N,1/350)
        peaks_index, properties = find_peaks(yf,height=-200,prominence=0.1)
        
        #for k in range(len(peaks_index)):
        #[print("%4.4f    \t %3.4f" %(freq[peaks_index[i]], properties['peak_heights'][i])) for i in range(len(peaks_index))]
        h_max_peak_1=np.unique(properties['peak_heights'])[-1]
        peak1_dimension = np.where(properties['peak_heights'] == h_max_peak_1)
        if (len(peak1_dimension[0])>1):
            index1 = np.where(properties['peak_heights'] == h_max_peak_1)[0][1]
        else:
            index1 = np.where(properties['peak_heights'] == h_max_peak_1)[0][0]

        h_max_peak_2=np.unique(properties['peak_heights'])[-2] 
        peak2_dimension = np.where(properties['peak_heights'] == h_max_peak_2)

        if (len(peak2_dimension[0])>1):
            index2 = np.where(properties['peak_heights'] == h_max_peak_2)[0][1]
        else:
            index2 = np.where(properties['peak_heights'] == h_max_peak_2)[0][0]

        val={'tempo':i,f'<>({V})':mean,f'var({V})':var,f'kurt({V})':kurt,f'RMS({V})':rms,f'freq1({V})':xf[index1],f'freq2({V})':xf[index2],f'amp1({V})':h_max_peak_1,f'amp2({V})':h_max_peak_2}
        tab = tab.astype("float64")
        tab=tab.append(val,ignore_index=True)
    return tab
    #display(tab)

Si richiama la funzione descritta sopra per ogni variabile in modo da avere le statistiche sia nel tempo che in frequenza

In [23]:
from functools import reduce
GyrX=time_freq_feat('GyrX')
GyrY=time_freq_feat('GyrY')
GyrZ=time_freq_feat('GyrZ')
AccX=time_freq_feat('AccX')	
AccY=time_freq_feat('AccY')
AccZ=time_freq_feat('AccZ')
Roll=time_freq_feat('Roll')
Pitch=time_freq_feat('Pitch')
Yaw=time_freq_feat('Yaw')
c_9 =time_freq_feat('C9')
c_10 =time_freq_feat('C10')
c_11 =time_freq_feat('C11')
c_12 =time_freq_feat('C12')
c_13 =time_freq_feat('C13')
c_14 =time_freq_feat('C14')

merged = [GyrX,GyrY,GyrZ,AccX,AccY,AccZ,Roll,Pitch,Yaw,c_9,c_10,c_11,c_12,c_13,c_14]
df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['tempo'],how='outer'), merged)
df_merged["Guasto"] = guasto

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
display(df_merged)

c:\Python310\lib\site-packages\scipy\signal\_peak_finding.py:261: ComplexWarning: Casting complex values to real discards the imaginary part
  value = np.asarray(value, order='C', dtype=np.float64)
C:\Users\chiar\AppData\Local\Temp\ipykernel_15212\2937120196.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tab=tab.append(val,ignore_index=True)
c:\Python310\lib\site-packages\scipy\signal\_peak_finding.py:261: ComplexWarning: Casting complex values to real discards the imaginary part
  value = np.asarray(value, order='C', dtype=np.float64)
C:\Users\chiar\AppData\Local\Temp\ipykernel_15212\2937120196.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tab=tab.append(val,ignore_index=True)
c:\Python310\lib\site-packages\scipy\signal\_peak_finding.py:261: ComplexWarning: Casting complex values to real discards 

,tempo,<>(GyrX),var(GyrX),kurt(GyrX),RMS(GyrX),freq1(GyrX),freq2(GyrX),amp1(GyrX),amp2(GyrX),<>(GyrY),var(GyrY),kurt(GyrY),RMS(GyrY),freq1(GyrY),freq2(GyrY),amp1(GyrY),amp2(GyrY),<>(GyrZ),var(GyrZ),kurt(GyrZ),RMS(GyrZ),freq1(GyrZ),freq2(GyrZ),amp1(GyrZ),amp2(GyrZ),<>(AccX),var(AccX),kurt(AccX),RMS(AccX),freq1(AccX),freq2(AccX),amp1(AccX),amp2(AccX),<>(AccY),var(AccY),kurt(AccY),RMS(AccY),freq1(AccY),freq2(AccY),amp1(AccY),amp2(AccY),<>(AccZ),var(AccZ),kurt(AccZ),RMS(AccZ),freq1(AccZ),freq2(AccZ),amp1(AccZ),amp2(AccZ),<>(Roll),var(Roll),kurt(Roll),RMS(Roll),freq1(Roll),freq2(Roll),amp1(Roll),amp2(Roll),<>(Pitch),var(Pitch),kurt(Pitch),RMS(Pitch),freq1(Pitch),freq2(Pitch),amp1(Pitch),amp2(Pitch),<>(Yaw),var(Yaw),kurt(Yaw),RMS(Yaw),freq1(Yaw),freq2(Yaw),amp1(Yaw),amp2(Yaw),<>(C9),var(C9),kurt(C9),RMS(C9),freq1(C9),freq2(C9),amp1(C9),amp2(C9),<>(C10),var(C10),kurt(C10),RMS(C10),freq1(C10),freq2(C10),amp1(C10),amp2(C10),<>(C11),var(C11),kurt(C11),RMS(C11),freq1(C11),freq2(C11),amp1(C11),amp2(C11),<>(C12),var(C12),kurt(C12),RMS(C12),freq1(C12),freq2(C12),amp1(C12),amp2(C12),<>(C13),var(C13),kurt(C13),RMS(C13),freq1(C13),freq2(C13),amp1(C13),amp2(C13),<>(C14),var(C14),kurt(C14),RMS(C14),freq1(C14),freq2(C14),amp1(C14),amp2(C14),Guasto
0,0.0,-0.041897,0.019825,-0.908719,0.146711,69.0,51.0,6.981244,3.069249,0.016826,0.003197,0.054297,0.058913,62.0,63.0,2.150326,2.103738,0.100116,0.006544,-1.639515,0.128640,52.0,64.0,0.589227,0.389142,-0.106334,0.046690,-0.346622,0.240550,93.0,95.0,12.340442,9.908352,0.349503,0.045756,-0.498059,0.409606,114.0,107.0,9.714730,7.895515,-10.694732,0.668859,-1.030573,10.725868,0.0,101.0,176.286128,18.423422,0.370085,1.060133,0.235831,1.092738,84.0,83.0,3.826128,3.641753,-0.162165,0.127906,0.802343,0.392224,0.0,64.0,43.702129,1.165395,14.909345,3.624475,-0.784801,15.030060,69.0,91.0,-1.984904,-1.993132,1560.065527,6445.432837,0.427352,1562.124041,90.0,89.0,686.042794,606.421664,1494.472934,3758.729980,0.049611,1495.726370,86.0,83.0,1364.425034,902.784964,1539.381766,5935.836695,0.600493,1541.303068,90.0,86.0,908.974150,577.556395,1516.940171,3714.022125,0.248885,1518.160375,94.0,91.0,1409.713230,451.412811,1517.626781,4734.171738,0.184531,1519.181269,100.0,97.0,955.855408,577.858608,1538.287749,5480.199821,-0.137235,1540.062916,107.0,108.0,1123.254317,867.146306,0
1,1.0,-0.042134,0.019788,-0.909437,0.146654,81.0,60.0,7.061480,3.105682,0.016689,0.003194,0.053572,0.058853,64.0,81.0,2.494792,1.887597,0.100326,0.006541,-1.638892,0.128792,50.0,62.0,0.558014,0.406045,-0.106958,0.046694,-0.348761,0.240835,93.0,95.0,12.523760,9.494893,0.349606,0.045629,-0.491289,0.409541,116.0,110.0,9.394187,6.554628,-10.693133,0.667853,-1.025377,10.724227,0.0,94.0,177.020256,18.538978,0.362443,1.077672,0.227383,1.098169,82.0,81.0,4.026074,3.828350,-0.159972,0.129236,0.737663,0.393013,0.0,68.0,44.047795,1.037362,14.921307,3.664517,-0.792659,15.043254,64.0,88.0,-2.004312,-2.021730,1560.215909,6435.030174,0.439141,1562.270921,86.0,85.0,703.510157,456.177176,1494.414773,3749.212089,0.053388,1495.665094,98.0,96.0,1380.857962,903.188845,1539.548295,5928.687120,0.611929,1541.467093,98.0,93.0,894.512191,517.509883,1516.869318,3705.207945,0.250825,1518.086694,103.0,100.0,1386.112682,449.198001,1517.562500,4722.138533,0.188000,1519.113118,98.0,95.0,997.015927,620.758972,1538.451705,5474.048943,-0.128855,1540.224706,111.0,112.0,1128.646163,873.959175,0
2,2.0,-0.042387,0.019755,-0.910883,0.146614,71.0,52.0,7.123181,3.241375,0.016595,0.003188,0.057820,0.058776,60.0,79.0,2.632545,1.889036,0.100523,0.006536,-1.637946,0.128926,52.0,66.0,0.440320,0.416940,-0.107145,0.046574,-0.341693,0.240670,93.0,95.0,12.545314,9.138572,0.350103,0.045586,-0.492431,0.409914,104.0,98.0,9.204193,5.372044,-10.691471,0.666931,-1.020329,10.722528,0.0,101.0,177.762811,18.668782,0.354844,1.094994,0.216474,1.103543,86.0,85.0,4.242015,4.007221,-0.157790,0.130548,0.674999,0.393797,0.0,72.0,44.383770,0.914040,14.933201,3.704047,-0.801227,15.056362,65.0,66.0,-1.038030,-2.072733

In [24]:
m1_nofault = df_merged

## Output

Salviamo la tabella ottenuta in un file csv

In [25]:
path_file = path_file.replace(r"M1\NO_FAULT1", "")
os.chdir(path_file)
df_merged.to_csv('m1_nofault.csv', index=False)
print(path_file)

c:\Users\chiar\OneDrive\Desktop\MantenzionePreventiva\ProgettoManutenzionePreventiva\
